# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298257578646                   -0.85    5.1         
  2   -8.300184627619       -2.72       -1.26    1.2   85.5ms
  3   -8.300428844987       -3.61       -1.89    3.9    260ms
  4   -8.300461936635       -4.48       -2.76    1.4   79.6ms
  5   -8.300464131230       -5.66       -3.07    2.5   98.2ms
  6   -8.300464395944       -6.58       -3.27   10.5    175ms
  7   -8.300464518221       -6.91       -3.42    2.5    107ms
  8   -8.300464583172       -7.19       -3.56    1.5    105ms
  9   -8.300464626169       -7.37       -3.76    1.4    118ms
 10   -8.300464637074       -7.96       -3.90    1.6   89.8ms
 11   -8.300464643303       -8.21       -4.20    1.9    102ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67463526331                   -0.70    6.8         
  2   -16.67870619579       -2.39       -1.14    1.4    201ms
  3   -16.67920998191       -3.30       -1.87    3.2    237ms
  4   -16.67927732114       -4.17       -2.76    3.6    261ms
  5   -16.67928587500       -5.07       -3.18    5.1    310ms
  6   -16.67928619958       -6.49       -3.48    2.6    228ms
  7   -16.67928621762       -7.74       -3.96    1.9    198ms
  8   -16.67928622145       -8.42       -4.58    3.1    248ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32690944438                   -0.56    7.0         
  2   -33.33427577073       -2.13       -1.00    1.0    670ms
  3   -33.33596939315       -2.77       -1.73    5.6    940ms
  4   -33.33618082228       -3.67       -2.64    5.0    828ms
  5   -33.33693219187       -3.12       -2.80    6.5    1.07s
  6   -33.33694125076       -5.04       -3.20    4.2    813ms
  7   -33.33694369114       -5.61       -3.95    3.6    943ms
  8   -33.33694378132       -7.04       -4.56    4.6    964ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298247978480                   -0.85    5.2         
  2   -8.300235126664       -2.70       -1.58    1.0   57.8ms
  3   -8.300421209904       -3.73       -2.50    3.1   82.2ms
  4   -8.300361592637   +   -4.22       -2.24    4.0    104ms
  5   -8.300464010278       -3.99       -3.42    1.1   66.4ms
  6   -8.300464558878       -6.26       -3.74    3.9    127ms
  7   -8.300464633104       -7.13       -4.13    1.4   78.2ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32586248849                   -0.56    6.9         
  2   -33.26708132861   +   -1.23       -1.24    1.6    594ms
  3   +22.57839765009   +    1.75       -0.19    8.2    1.56s
  4   -33.32125474071        1.75       -1.66    6.0    1.39s
  5   -32.99135087521   +   -0.48       -1.20    3.9    970ms
  6   -30.52561759969   +    0.39       -0.84    5.0    1.02s
  7   -33.31810387581        0.45       -1.84    4.9    992ms
  8   -33.32945676824       -1.94       -2.00    2.0    716ms
  9   -33.33539782111       -2.23       -2.15    2.4    762ms
 10   -33.33597165611       -3.24       -2.55    1.8    688ms
 11   -33.33675402268       -3.11       -2.87    2.2    694ms
 12   -33.33685146084       -4.01      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.